In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random



In [2]:
#讀pgm檔案轉成array
def pgmtoarray(pgm):
    img   = cv2.imread(pgm,0)#讀取pgm
    array = np.reshape( img, (img.size,1) ) # 10304*1 array 
    return array

In [3]:
orlpath=r"D:\Desktop\peopletest" #path

#讀入人臉庫，每個人隨機選擇 k(k=5) 張照片(pgm檔案)作為訓練集，其餘作為測試集
def load_orl(k):
    
    train_face   = np.zeros( (92*112 , 40*k) )     #訓練的陣列，10304*200 (填0)
    train_label  = np.zeros(  40*k  )              #200   (填0)，label的部分
    
    test_face    = np.zeros( (92*112 , 40*(10-k)) )#測試的陣列，10304*200 (填0)
    test_label   = np.zeros(  40*(10-k) )          #200   (填0)，label的部分

    
    a = random.sample(range(1,11),10) #每個人照片10張，隨機打散作為a列表，然後前五個做為train，後五個做為test
    
    for i in range(40):      #共有40个人
        people_num = i+1     #人的label ( s1-> people_num=1 )
        for j in range(10):  #每个人都有10张照片
            
            #path = D:\Desktop\peopletest\s{}.pgm
            image = orlpath + '/s' + str(people_num) + '/' + str(a[j]) + '.pgm'
            
            img   = pgmtoarray(image)  #讀取pgm檔案並轉換成陣列 
            
            #前五個做為訓練集，後五個做為測試集
            if j < k:              
                train_face[:,i*k+j]         = img[:,0]  
                train_label[i*k+j]          = people_num     
            else:        
                test_face[:,i*(10-k)+(j-k)] = img[:,0]
                test_label[i*(10-k)+(j-k)]  = people_num
                
    return train_face, train_label, test_face, test_label


In [4]:
#PCA             
def PCA(data,r):
    data       = np.float32(np.mat(data))
    
    rows,cols  = np.shape(data)                   #10304 * 200

    data_mean  = np.mean(data,1)                  #對欄求平均值 
    
    A          = data-np.tile(data_mean,(1,cols)) #將所有的樣本減去平均值得到A  (x-mu)
 
    C          = A.T * A                          #協方差矩陣  
    
    D , V      = np.linalg.eig(C)                 #求協方差矩陣的特徵值和特徵向量 200   200*200
    
    V_r        = V[:,0:r]                         #按列取前r个特征向量 200*(10~50)
    
    V_r        = A * V_r                          #10304*(10~50)
    
      
    for i in range(r):
        V_r[i,:] = V_r[i,:] / np.linalg.norm(V_r[i,:])#特徵向量歸一化 10304*(10~50)
        
    final_data = A.T * V_r

    return final_data, data_mean, V_r

In [5]:
def lda(data, label, dim): # data_train_new 200 * (10~50) , train_label(200,), r1[i] (10~50)
    
    clusters = np.unique(label) #沒有重複的 label: 1~40  
    dimension = data.shape[1] #r1[i] 10~50
    
    #within_class scatter matrix
    Sw = np.zeros( (dimension,dimension) ) 
    for i in clusters: #每個標籤 1~40
        datai       =   data[label == i]                 #屬於i標籤(人)的data   5 * (10~50)
        datai       =   datai - datai.mean(0)            #計算列的平均  datai = x-mu 
        Swi         =   np.mat(datai).T * np.mat(datai)  #mat函數用於  將輸入視為矩陣 x-mu.T * (x-mu)
        Sw         +=   Swi

        
    #between_class scatter matrix
    SB = np.zeros( (dimension,dimension) )
    u  = data.mean(0)  #所有樣本的列的平均值
    for i in clusters:
        Ni           =   data[label == i].shape[0] #5張照片
        ui           =   data[label == i].mean(0)  #某個類別的平均值
        SBi          =   Ni * np.mat(ui - u).T * np.mat(ui - u)  #   Ni * (ui - u) * (ui - u).T
        SB          +=   SBi
        

    S                =   np.linalg.inv(Sw) * SB #Sw反矩陣 * SB矩陣
    
    eigVals,eigVects =   np.linalg.eig(S)       #求特徵值、特徵向量  eigVals:(10~50),   eigVects:(10~50),(10~50)  

    eigValInd        =   np.argsort(eigVals)    #排序 (10~50), 
    
    w                =   eigVects[:,eigValInd]  

    data_ndim        =   np.dot(data, w)
    
    return data_ndim, w


In [6]:
#人臉辨識
def face_rec():
    r1=[10,20,30,40,50] #維度 10~50
    
    for i in range(5):   
        print("當降維到%d時"%(r1[i]))
        
        train_face, train_label, test_face, test_label =  load_orl(5)#得到數據集(分成一半 k=5)
        
        #利用PCA算法進行訓練
        data_train_new,  data_mean,  V_r = PCA( train_face , r1[i] )    
        
        num_train       = data_train_new.shape[0]                     # 訓練數量 200
        num_test        = test_face.shape[1]                          # 测试數量 200

        temp_face       = test_face - np.tile(data_mean, (1,num_test))# test_x-test_mu
        data_test_new   = temp_face.T*V_r                             # y=A.T * x-mu
        
        data_test_new   = np.array( data_test_new )                   # mat change to array 200*10
        data_train_new  = np.array( data_train_new )                  # 200*10
        
        #測試準確度 KNN (距離計算)
        true_num        = 0                      # TP (正確分辨的)
        matrix          = np.zeros( (41 , 41) )  # confuse matrix
        no              = 0                      # 非TP的 
        
        for n in range(num_test):     # test 數據集(200張)
            
            testFace           = data_test_new[n,:]                                #new array
            
            diffMat            = data_train_new - np.tile(testFace,(num_train,1))  #訓練數據與測試臉之間距離
            
            sqDiffMat          = diffMat**2
            
            sqDistances        = sqDiffMat.sum(axis=1)                             #按欄求和
            
            sortedDistIndicies = sqDistances.argsort()                             #對向量從小到大排序，使用的是索引值,得到一個向量
            
            indexMin           = sortedDistIndicies[0]                             #距離最近的索引

            
            if train_label[indexMin] == test_label[n]:#True and Positive (TP)  
                a            =  train_label[indexMin].astype(int) #train_label
                b            =  test_label[n].astype(int)         #test_label
                true_num     += 1                                 #TP數量++
                matrix[a,b]  += 1                                 #confuse matrix 更新
                
            else:                                     #非TP的
                a            =  train_label[indexMin].astype(int) #train_label
                b            =  test_label[n].astype(int)         #test_label
                no           += 1                                 #非TP的數量++
                matrix[a,b]  += 1                                 #confuse matrix 更新
               
        total = 0                   #確認數量
        for m in range(41):
            total = total + matrix[m,m]
    
        accuracy = float(true_num) / num_test
        
        print("number of not correct:", no)
        print("number of correct:"    , true_num)
        print("PCA accuracy:"          , accuracy * 100)
        print("confuse matrix:\n"     , matrix)
    
        print("----------------------") #lda
        
        #利用LDA算法進行訓練
        lda_data_train , W = lda(data_train_new, train_label, r1[i])  
        
        num_train          = lda_data_train.shape[0]                     # 訓練數量 200
        num_test           = test_face.shape[1]                          # 测试數量 200
        
        lda_data_test      = np.dot(data_test_new,W)                     # 得到测试脸在特征向量下的数据  y = X * W
        
        lda_data_test      = np.array( lda_data_test )                   # mat change to array 200*(10~50)
        lda_data_train     = np.array( lda_data_train )                  # 200*(10~50)
        
        #測試準確度 KNN (距離計算)
        true_num1 = 0
        matrix1   = np.zeros( (41 , 41) )
        no1       = 0
        
        for n in range(num_test):
            testFace1           = lda_data_test[n,:]

            diffMat1            = lda_data_train - np.tile(testFace1,(num_train,1))

            sqDiffMat1          = diffMat1**2
            
            sqDistances1        = sqDiffMat1.sum(axis=1)#按欄求和
            
            sortedDistIndicies1 = sqDistances1.argsort()#對向量從小到大排序，使用的是索引值,得到一個向量
            
            indexMin            = sortedDistIndicies1[0]#

            
            if train_label[indexMin] == test_label[n]:
                a            =  train_label[indexMin].astype(int)
                b            =  test_label[n].astype(int)
                true_num1    += 1
                matrix1[a,b] += 1
            else:
                a            =  train_label[indexMin].astype(int)
                b            =  test_label[n].astype(int)
                no1          += 1
                matrix1[a,b] += 1
               
        total1 = 0        
        for m in range(41):
            total1 = total1 + matrix1[m,m]
            
        accuracy1 = float(true_num1) / num_test
        
        print("number of not correct:", no1)
        print("number of correct:"    , true_num1)
        print("LDA accuracy:"         , accuracy1 * 100)
        print("confuse matrix:\n"     , matrix1)
        
        print("-------------------------------------------------\n")

        
if __name__ =='__main__':  
    face_rec() 

當降維到10時
number of not correct: 22
number of correct: 178
PCA accuracy: 89.0
confuse matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 3. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 5. 0. 0.]
 [0. 0. 0. ... 0. 5. 0.]
 [0. 0. 0. ... 0. 0. 5.]]
----------------------
number of not correct: 22
number of correct: 178
LDA accuracy: 89.0
confuse matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 3. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 5. 0. 0.]
 [0. 0. 0. ... 0. 5. 0.]
 [0. 0. 0. ... 0. 0. 5.]]
-------------------------------------------------

當降維到20時
number of not correct: 18
number of correct: 182
PCA accuracy: 91.0
confuse matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 5. 0. 0.]
 [0. 0. 0. ... 0. 5. 0.]
 [0. 0. 0. ... 0. 0. 3.]]
----------------------
number of not correct: 18
number of correct: 182
LDA accuracy: 91.0
confuse matrix:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]
 ...